##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*. 

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

Install and import the Keras Tuner.

In [2]:
#!pip install -q -U keras-tuner
import kerastuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). 

Load the data.

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined `HyperModel` classes - [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperxception-class) and [HyperResNet](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Before running the hyperparameter search, define a callback to clear the training outputs at the end of every training step.

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 96 and the optimal learning rate for the optimizer
is 0.001.



To finish this tutorial, retrain the model with the optimal hyperparameters from the search.

In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - ETA: 12:29 - loss: 2.5533 - accuracy: 0.0000e+0 - ETA: 23s - loss: 1.7620 - accuracy: 0.3544      - ETA: 13s - loss: 1.5081 - accuracy: 0.454 - ETA: 9s - loss: 1.3610 - accuracy: 0.509 - ETA: 7s - loss: 1.2615 - accuracy: 0.54 - ETA: 6s - loss: 1.1883 - accuracy: 0.57 - ETA: 5s - loss: 1.1318 - accuracy: 0.59 - ETA: 5s - loss: 1.0923 - accuracy: 0.60 - ETA: 5s - loss: 1.0568 - accuracy: 0.62 - ETA: 4s - loss: 1.0354 - accuracy: 0.63 - ETA: 4s - loss: 1.0132 - accuracy: 0.63 - ETA: 4s - loss: 0.9970 - accuracy: 0.64 - ETA: 4s - loss: 0.9785 - accuracy: 0.65 - ETA: 4s - loss: 0.9596 - accuracy: 0.65 - ETA: 4s - loss: 0.9431 - accuracy: 0.66 - ETA: 3s - loss: 0.9283 - accuracy: 0.66 - ETA: 3s - loss: 0.9129 - accuracy: 0.67 - ETA: 3s - loss: 0.9006 - accuracy: 0.67 - ETA: 3s - loss: 0.8874 - accuracy: 0.68 - ETA: 3s - loss: 0.8757 - accuracy: 0.68 - ETA: 3s - loss: 0.8654 - accuracy: 0.69 - ETA: 3s - loss: 0.8547 - accuracy: 0.69 - E

1875/1875 [==============================] - ETA: 7s - loss: 0.2720 - accuracy: 0.96 - ETA: 2s - loss: 0.3807 - accuracy: 0.87 - ETA: 2s - loss: 0.3694 - accuracy: 0.87 - ETA: 2s - loss: 0.3619 - accuracy: 0.87 - ETA: 2s - loss: 0.3581 - accuracy: 0.87 - ETA: 2s - loss: 0.3560 - accuracy: 0.87 - ETA: 2s - loss: 0.3556 - accuracy: 0.87 - ETA: 2s - loss: 0.3555 - accuracy: 0.87 - ETA: 2s - loss: 0.3552 - accuracy: 0.86 - ETA: 2s - loss: 0.3552 - accuracy: 0.86 - ETA: 2s - loss: 0.3551 - accuracy: 0.86 - ETA: 2s - loss: 0.3548 - accuracy: 0.86 - ETA: 2s - loss: 0.3545 - accuracy: 0.86 - ETA: 2s - loss: 0.3541 - accuracy: 0.86 - ETA: 2s - loss: 0.3536 - accuracy: 0.87 - ETA: 2s - loss: 0.3531 - accuracy: 0.87 - ETA: 2s - loss: 0.3528 - accuracy: 0.87 - ETA: 2s - loss: 0.3525 - accuracy: 0.87 - ETA: 2s - loss: 0.3522 - accuracy: 0.87 - ETA: 2s - loss: 0.3519 - accuracy: 0.87 - ETA: 2s - loss: 0.3517 - accuracy: 0.87 - ETA: 2s - loss: 0.3515 - accuracy: 0.87 - ETA: 2s - loss: 0.3513 - accura

1875/1875 [==============================] - ETA: 7s - loss: 0.1808 - accuracy: 0.90 - ETA: 3s - loss: 0.2888 - accuracy: 0.90 - ETA: 2s - loss: 0.2796 - accuracy: 0.90 - ETA: 2s - loss: 0.2762 - accuracy: 0.90 - ETA: 2s - loss: 0.2763 - accuracy: 0.90 - ETA: 2s - loss: 0.2775 - accuracy: 0.90 - ETA: 2s - loss: 0.2790 - accuracy: 0.89 - ETA: 2s - loss: 0.2808 - accuracy: 0.89 - ETA: 2s - loss: 0.2822 - accuracy: 0.89 - ETA: 2s - loss: 0.2832 - accuracy: 0.89 - ETA: 2s - loss: 0.2837 - accuracy: 0.89 - ETA: 2s - loss: 0.2841 - accuracy: 0.89 - ETA: 2s - loss: 0.2844 - accuracy: 0.89 - ETA: 2s - loss: 0.2845 - accuracy: 0.89 - ETA: 2s - loss: 0.2844 - accuracy: 0.89 - ETA: 1s - loss: 0.2843 - accuracy: 0.89 - ETA: 1s - loss: 0.2841 - accuracy: 0.89 - ETA: 1s - loss: 0.2838 - accuracy: 0.89 - ETA: 1s - loss: 0.2836 - accuracy: 0.89 - ETA: 1s - loss: 0.2833 - accuracy: 0.89 - ETA: 1s - loss: 0.2831 - accuracy: 0.89 - ETA: 1s - loss: 0.2829 - accuracy: 0.89 - ETA: 1s - loss: 0.2828 - accura

1875/1875 [==============================] - ETA: 5s - loss: 0.1011 - accuracy: 0.96 - ETA: 2s - loss: 0.2532 - accuracy: 0.89 - ETA: 2s - loss: 0.2583 - accuracy: 0.89 - ETA: 2s - loss: 0.2603 - accuracy: 0.89 - ETA: 2s - loss: 0.2627 - accuracy: 0.89 - ETA: 2s - loss: 0.2649 - accuracy: 0.90 - ETA: 2s - loss: 0.2650 - accuracy: 0.90 - ETA: 2s - loss: 0.2647 - accuracy: 0.90 - ETA: 2s - loss: 0.2644 - accuracy: 0.90 - ETA: 2s - loss: 0.2640 - accuracy: 0.90 - ETA: 2s - loss: 0.2632 - accuracy: 0.90 - ETA: 2s - loss: 0.2626 - accuracy: 0.90 - ETA: 2s - loss: 0.2617 - accuracy: 0.90 - ETA: 2s - loss: 0.2607 - accuracy: 0.90 - ETA: 2s - loss: 0.2598 - accuracy: 0.90 - ETA: 2s - loss: 0.2590 - accuracy: 0.90 - ETA: 2s - loss: 0.2585 - accuracy: 0.90 - ETA: 2s - loss: 0.2582 - accuracy: 0.90 - ETA: 1s - loss: 0.2578 - accuracy: 0.90 - ETA: 1s - loss: 0.2576 - accuracy: 0.90 - ETA: 1s - loss: 0.2575 - accuracy: 0.90 - ETA: 1s - loss: 0.2574 - accuracy: 0.90 - ETA: 1s - loss: 0.2572 - accura

The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite = True` argument while instantiating the tuner.

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.